In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
%load_ext line_profiler
%load_ext memory_profiler
%load_ext Cython

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed



In [5]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]


In [6]:
session_settings = [{
                                    'decision': '10:40',
                                    'dt': datetime(1900, 1, 1),
                                    'execution': '10:45',
                                    'start': '00:32'},

                                {
                                    'decision': '11:40',
                                    'dt': datetime(2011, 9, 30),
                                    'execution': '11:45',
                                    'start': '01:32'},

                                {
                                    'decision': '12:40',
                                    'dt': datetime(2012, 1, 1),
                                    'execution': '12:45',
                                    'start': '02:32'},
                                ]
tz = pytz.timezone('US/Pacific')
sess = AssetSession(session_settings, tz)

In [7]:
feed = DataFeed()

In [8]:
def load_case1():
    dframes_list = []
    for data in mongo_db['quotes_intraday'].find({'tckr': 'US.F.CL.Q12.120720'}):
        df = pickle.loads(data['ohlc'])
        dframes_list.append(df)    

    result = pd.concat(dframes_list)

In [9]:
def load_case2():
    result_df = pd.DataFrame()
    for data in mongo_db['quotes_intraday'].find({'tckr': 'US.F.CL.Q12.120720'}):
        df = pickle.loads(data['ohlc'])
        result_df = pd.concat([result_df, df])    

    return result_df

In [10]:
%timeit load_case1()

10 loops, best of 3: 162 ms per loop


In [11]:
%timeit load_case2()

1 loop, best of 3: 341 ms per loop


In [ ]:
%lprun -f load_case1 load_case1()

In [18]:
%lprun -f load_case2 load_case2()

In [15]:
df = load_case2()
df.tz_convert('US/Pacific', copy=False)

,o,h,l,c,v
dt,,,,,
2011-07-11 11:29:00-07:00,100.26,100.26,100.26,100.26,0.0
2011-07-12 11:29:00-07:00,101.88,101.88,101.88,101.88,0.0
2011-07-13 11:29:00-07:00,102.70,102.70,102.70,102.70,0.0
2011-07-14 08:44:00-07:00,101.31,101.31,101.31,101.31,1.0
2011-07-14 08:50:00-07:00,101.30,101.30,101.30,101.30,1.0
2011-07-14 09:34:00-07:00,100.89,100.89,100.89,100.89,1.0
2011-07-14 10:31:00-07:00,100.51,100.61,100.51,100.61,3.0
2011-07-14 10:33:00-07:00,100.43,100.43,100.30,100.30,3.0
2011-07-14 10:34:00-07:00,100.25,100.25,100.25,100.25,1.0


In [28]:
%mprun -f df.tz_convert df.tz_convert('US/Pacific', copy=True)

In [13]:
dt1 = df.index[-1]


In [14]:
%lprun -f sess.date_is_insession df.index.map(lambda x: sess.date_is_insession(x))

In [14]:
%lprun -f sess.date_is_insession sess.date_is_insession(dt1)

In [17]:
%timeit df.between_time('9:30', '10:45')

100 loops, best of 3: 5.39 ms per loop


In [15]:
%timeit df.index.map(lambda x: sess.date_is_insession(x))

1 loop, best of 3: 1.22 s per loop


In [25]:
%mprun -f sess.filter_dataframe sess.filter_dataframe(df)

In [23]:
%timeit sess.filter_dataframe(df)

100 loops, best of 3: 8.92 ms per loop


In [19]:
date_start = datetime(2012, 1, 1)
date_end = datetime(2013, 1, 1)

%timeit df[(df.index >= date_start) & (df.index < date_end)]

100 loops, best of 3: 3.49 ms per loop


In [22]:
%timeit df.ix[date_start:date_end]

1000 loops, best of 3: 504 µs per loop


In [49]:
pd.Timestamp(np_date).date()

datetime.date(2011, 7, 11)

In [28]:
def datetime64_to_time_of_day(datetime64_array):
    """
    Return a new array. For every element in datetime64_array return the time of day (since midnight).
    >>> datetime64_to_time_of_day(np.array(['2012-01-02T01:01:01.001Z'],dtype='datetime64[ms]'))
    array([3661001], dtype='timedelta64[ms]')
    >>> datetime64_to_time_of_day(np.datetime64('2012-01-02T01:01:01.001Z','[ms]'))
    numpy.timedelta64(3661001,'ms')
    """
    day = datetime64_array.astype('datetime64[D]').astype(datetime64_array.dtype)
    time_of_day = datetime64_array - day
    return time_of_day

In [30]:
%timeit datetime64_to_time_of_day(df.index.values)

100 loops, best of 3: 3.88 ms per loop


In [40]:
(datetime(2010, 1, 1, 18, 29) - datetime(2010, 1, 1)).seconds * 1000000000

66540000000000

In [32]:
 datetime64_to_time_of_day(df.index.values)

array([66540000000000, 66540000000000, 66540000000000, ..., 66420000000000,
       66480000000000, 66540000000000], dtype='timedelta64[ns]')

In [41]:
np.datetime64(datetime(2011, 1,1))

numpy.datetime64('2011-01-01T00:00:00.000000')

In [119]:
sess_params = [x for x in sess.sessions]

In [120]:
sess_params

[{'decision': datetime.time(10, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(1900, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(10, 45, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'start': datetime.time(0, 32, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>)},
 {'decision': datetime.time(11, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(2011, 9, 30, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(11, 45, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'start': datetime.time(1, 32, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>)},
 {'decision': datetime.time(12, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(2012, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(12, 45, tz

In [78]:
%%cython -a
import numpy as np
import pandas as pd
from datetime import datetime

def filter_index(self, dataframe_index, sess_params):
    """
    Creates boolean filter array used to filter dataframe from out-of-session datapoints
    :param dataframe_index:
    :return:
    """

    flt = np.empty(len(dataframe_index))
    flt.fill(False)
    start_time = 0
    end_time = 0
    next_sess_date = 0

    
    date_array = dataframe_index.values
    time_array = date_array.view('datetime64[D]')
    
    date = date_array[0]
    for sess_idx, sess in enumerate(reversed(sess_params)):
        if date >= sess['dt']:
            start_datetime = np.datetime64(datetime.combine(date, sess['start']))
            end_datetime = np.datetime64(datetime.combine(date, sess['decision']))
            
            

            if sess_idx > 0:
                next_sess_date = np.datetime64(sess_params[len(sess_params) - sess_idx]['dt'])
            else:
                next_sess_date = 0
    
        
    for i in range(len(dataframe_index)):
        dt = date_array[i]
        t = time_array[i]

        if t >= start_time and t <= end_time:
            flt[i] = 1

    return flt

In [64]:
sess_params[0]['start']

ValueError: Could not convert object to NumPy datetime

In [74]:
%timeit df.index.values.astype('datetime64[D]')

1000 loops, best of 3: 1.94 ms per loop


In [75]:
%timeit df.index.values.view('datetime64[D]')

The slowest run took 11.83 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.48 µs per loop


In [79]:
dt = df.index.values

In [83]:
date_arr = dt.astype('datetime64[D]').view('uint64')

In [84]:
date_arr

array([15166, 15167, 15168, ..., 15541, 15541, 15541], dtype=uint64)

In [85]:
date_arr2 = dt.view('uint64')

In [86]:
date_arr2

array([1310408940000000000, 1310495340000000000, 1310581740000000000, ...,
       1342808820000000000, 1342808880000000000, 1342808940000000000], dtype=uint64)

In [95]:
datetime.fromtimestamp(, tz=pytz.timezone("UTC"))

datetime.datetime(2011, 7, 12, 18, 29, tzinfo=<UTC>)

In [123]:
sess_params

[{'decision': datetime.time(10, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(1900, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(10, 45, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'start': datetime.time(0, 32, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>)},
 {'decision': datetime.time(11, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(2011, 9, 30, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(11, 45, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'start': datetime.time(1, 32, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>)},
 {'decision': datetime.time(12, 40, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'dt': datetime.datetime(2012, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(12, 45, tz

In [132]:
def sess_filt():
    df_list = []
    for i in range(1, len(sess_params)):
        if i < len(sess_params)-1:
            date_start = sess_params[i-1]['dt']
            date_end = sess_params[i]['dt']
            time_start = sess_params[i-1]['start']
            time_end = sess_params[i-1]['decision']
        else:
            date_start = sess_params[i-1]['dt']        
            date_end = datetime(2100, 1, 1)        
            time_start = sess_params[i-1]['start']
            time_end = sess_params[i-1]['decision']

        print('DateStart: {0} DateEnd: {1}'.format(date_start, date_end))
        print('TimeStart: {0} TimeEnd: {1}'.format(time_start, time_end))
        tmp_df = df[(df.index >= date_start) & (df.index < date_end)]
        df_list.append(tmp_df.between_time(time_start, time_end))

    df_sess = pd.concat(df_list)

In [133]:
%timeit sess_filt()

DateStart: 1900-01-01 00:00:00-07:53 DateEnd: 2011-09-30 00:00:00-07:53
TimeStart: 00:32:00 TimeEnd: 10:40:00
DateStart: 2011-09-30 00:00:00-07:53 DateEnd: 2100-01-01 00:00:00
TimeStart: 01:32:00 TimeEnd: 11:40:00
DateStart: 1900-01-01 00:00:00-07:53 DateEnd: 2011-09-30 00:00:00-07:53
TimeStart: 00:32:00 TimeEnd: 10:40:00
DateStart: 2011-09-30 00:00:00-07:53 DateEnd: 2100-01-01 00:00:00
TimeStart: 01:32:00 TimeEnd: 11:40:00
DateStart: 1900-01-01 00:00:00-07:53 DateEnd: 2011-09-30 00:00:00-07:53
TimeStart: 00:32:00 TimeEnd: 10:40:00
DateStart: 2011-09-30 00:00:00-07:53 DateEnd: 2100-01-01 00:00:00
TimeStart: 01:32:00 TimeEnd: 11:40:00
DateStart: 1900-01-01 00:00:00-07:53 DateEnd: 2011-09-30 00:00:00-07:53
TimeStart: 00:32:00 TimeEnd: 10:40:00
DateStart: 2011-09-30 00:00:00-07:53 DateEnd: 2100-01-01 00:00:00
TimeStart: 01:32:00 TimeEnd: 11:40:00
DateStart: 1900-01-01 00:00:00-07:53 DateEnd: 2011-09-30 00:00:00-07:53
TimeStart: 00:32:00 TimeEnd: 10:40:00
DateStart: 2011-09-30 00:00:00-07:

In [135]:
df.index.tz

<UTC>